In [1]:
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

2023-06-06 23:23:05.666927: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 23:23:05.698120: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 23:23:05.698499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 23:23:06.515106: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!ls

In [ ]:
!python model_main_tf2.py --model_dir=output/ --pipeline_config_path=ssd_mobilenet_v2_ft/pipeline.config

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ssd_mobilenet_v2/pipeline.config --trained_checkpoint_dir src/output/ --output_directory src/ssd_mobilenet_v2_ft2

In [2]:
PATH_TO_MODEL_DIR = 'efficientnet512_ft/'
PATH_TO_LABELS = '/home/irizqy/ml_ws/bangkit-ws/data/label_map.pbtxt'

In [3]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

2023-06-06 23:23:10.552291: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-06 23:23:10.553132: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Done! Took 18.03534507751465 seconds


In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [12]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import os


rand_im = np.random.choice(os.listdir('/home/irizqy/ml_ws/bangkit-ws/data/test'), 1)
# rand_im = '/home/irizqy/ml_ws/bangkit-ws/data/test/2_20230507_092258.jpg'

BASED_PATH = '/home/irizqy/ml_ws/bangkit-ws/data/test'
IMAGE_PATHS = [os.path.join(BASED_PATH, im) for im in rand_im]
IMAGE_PATHS

warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
count = 0


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)
    
    if image_np.shape[2] == 4:
        image_np = image_np[:, :, :3]

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    # print(detections)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.35,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    im = Image.fromarray(image_np_with_detections)
    im.save(f"detected-logo2{count}.jpeg")
    count+=1
    print('Done')
    plt.show()

# sphinx_gallery_thumbnail_number = 2

Running inference for /home/irizqy/ml_ws/bangkit-ws/data/test/25_20230507_092258.jpg... defaultdict(<class 'str'>, {(0.3572155833244324, 0.12918847799301147, 0.5184792280197144, 0.3359139561653137): 'Chartreuse', (0.34423086047172546, 0.36981093883514404, 0.5366073846817017, 0.8243287801742554): 'Chartreuse', (0.6441798806190491, 0.2819415330886841, 0.699314296245575, 0.4230361580848694): 'Chartreuse'})
Done


In [13]:
import cv2 as cv

raw_detection_boxes <br>
detection_scores<br>
detection_boxes<br>
detection_anchor_indices<br>
raw_detection_scores<br>
detection_classes<br>
detection_multiclass_scores<br>
num_detections

In [18]:
print(detections['detection_boxes'])

[[0.35721558 0.12918848 0.5184792  0.33591396]
 [0.34423086 0.36981094 0.5366074  0.8243288 ]
 [0.6441799  0.28194153 0.6993143  0.42303616]
 [0.32009304 0.12494234 0.6274737  0.31551147]
 [0.2835581  0.         0.5200812  0.29064035]
 [0.27727234 0.08595228 0.54685825 0.3574895 ]
 [0.3812036  0.19774021 0.52450585 0.31444985]
 [0.6627163  0.26490697 0.7153736  0.4359059 ]
 [0.6402293  0.27458945 0.7016925  0.34304735]
 [0.34428847 0.34047174 0.59808606 0.6544788 ]
 [0.6543159  0.2988727  0.7170821  0.3742255 ]
 [0.0527156  0.         0.48652053 0.8403914 ]
 [0.22740021 0.         0.5075878  0.09078757]
 [0.01471399 0.         0.14344503 0.24430864]
 [0.6704098  0.29980212 0.72714114 0.47119194]
 [0.32480323 0.1391465  0.5553546  0.6912268 ]
 [0.37108344 0.11220341 0.48924035 0.24329865]
 [0.35051182 0.24125478 0.5135213  0.32673106]
 [0.5978304  0.30074722 0.6770557  0.44694722]
 [0.3746947  0.17717218 0.44957983 0.29822165]
 [0.39216056 0.38429466 0.6587703  0.888286  ]
 [0.31425267 

In [14]:
width, height, _ = image_np.shape
dets_index = np.where(detections['detection_scores'] >= .5)[0]
dets_index

array([0, 1, 2])

In [15]:
for i in dets_index:
    ymin, xmin, ymax, xmax = detections['detection_boxes'][i]
    (left, right, top, bottom) = (xmin*width, xmax*width, ymin*height, ymax*height)
    # print(int(left), int(right), int(top), int(bottom))
    # im = cv.rectangle(image_np, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 3)
    # Image.fromarray(im).save('test.jpg')
    cropped_im = image_np[int(top):int(bottom), int(left):int(right)]
    Image.fromarray(cropped_im).save(f'cropped-logo{i}.jpg')